<a href="https://colab.research.google.com/github/buterajacques1/alu-machine_learning/blob/main/Transfer_Learning/Transfer_Learning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This project applies transfer learning on the Fashion-MNIST dataset using three pre-trained models: VGG16, ResNet50, and InceptionV3. The goal is to classify images of fashion items into one of ten categories.**


***Step 1: Preparing the Dataset***

In [4]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load the Fashion-MNIST dataset
from tensorflow.keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the images to [0, 1] range and convert to 3 channels
x_train = np.stack([x_train]*3, axis=-1)
x_test = np.stack([x_test]*3, axis=-1)

# Resize the images to 32x32
x_train = np.array([np.pad(image, ((2, 2), (2, 2), (0, 0)), mode='constant') for image in x_train])
x_test = np.array([np.pad(image, ((2, 2), (2, 2), (0, 0)), mode='constant') for image in x_test])

# Convert labels to categorical format
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create data generators
train_datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
validation_generator = ImageDataGenerator().flow(x_test, y_test, batch_size=32)


**Step 2: Fine-Tuning the Pre-trained Models**

**VGG16**

In [5]:
# Load the VGG16 model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base layers
for layer in vgg_base.layers:
    layer.trainable = False

# Add custom layers on top
x = vgg_base.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

# Create the final model
vgg_model = Model(inputs=vgg_base.input, outputs=predictions)

# Compile the model
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
vgg_model.fit(train_generator, epochs=5, validation_data=validation_generator)


58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 910s 484ms/step - loss: 0.9656 - accuracy: 0.6743 - val_loss: 0.6333 - val_accuracy: 0.7751
Epoch 2/5
1875/1875 [==============================] - 898s 479ms/step - loss: 0.7957 - accuracy: 0.7112 - val_loss: 0.5810 - val_accuracy: 0.7913
Epoch 3/5
1875/1875 [==============================] - 919s 490ms/step - loss: 0.7670 - accuracy: 0.7217 - val_loss: 0.5613 - val_accuracy: 0.7982
Epoch 4/5
1875/1875 [==============================] - 896s 478ms/step - loss: 0.7473 - accuracy: 0.7283 - val_loss: 0.5634 - val_accuracy: 0.7983
Epoch 5/5
1875/1875 [==============================] - 916s 488ms/step - loss: 0.7364 - accuracy: 0.7309 - val_loss: 0.5648 - val_accuracy: 0.7996
